# 3章 顧客の全体像を把握する１０本ノック

ここでは、スポーツジムの会員データを使って顧客の行動を分析していきます。  
これまでと同様にまずはデータを理解し、加工した後、  
顧客の行動データを分析していきましょう。  
ここでは、機械学習に向けての初期分析を行います。

### ノック21：データを読み込んで把握しよう

目的：退会ユーザーと継続ユーザーの違いを把握する

手段
1,データを読み込んで把握する
ーーーー
2,顧客データの整形(バラバラのデータを結合して集計しやすくする)
3,顧客データの基礎集計(会員クラス,入った時のキャンペーン,男女比,在籍してる人と退会している人の数)
4,最新月の顧客データの基礎集計
→分析もまずはミニマムスタート。全体の基礎集計と比較して代わり映えがなかったので、利用履歴データを活用していくことに
ーーーー
5,利用履歴データの平均値、中央値、最大値、最小値を集計
6,利用履歴データから、最大値が4以上の場合はフラグを立てておく
ーーーー
7,顧客データと利用履歴データを結合
8,会員期間の計算
ーーーーー
9,顧客行動の統計量を把握(まずは全体の統計量、次にフラグを集計、最後に会員期間分布)
10,退会ユーザーと継続ユーザーの違いの把握(行動データ、期間、在籍期間ごとに深掘り)

In [ ]:
import pandas as pd
uselog = pd.read_csv('use_log.csv')
print(len(uselog))
uselog.head()

In [ ]:
customer = pd.read_csv('customer_master.csv')
print(len(customer))
customer.head()

In [ ]:
class_master = pd.read_csv('class_master.csv')
print(len(class_master))
class_master.head()

In [ ]:
campaign_master = pd.read_csv('campaign_master.csv')
print(len(campaign_master))
campaign_master.head()

### ノック22：顧客データを整形しよう

In [ ]:
customer_join = pd.merge(customer, class_master, on="class", how="left")
customer_join = pd.merge(customer_join, campaign_master, on="campaign_id", how="left")
customer_join.head()

In [ ]:
customer_join.isnull().sum()

In [ ]:
print(len(customer))
print(len(customer_join))

### ノック23：顧客データの基礎集計をしよう

In [ ]:
customer_join.groupby("class_name").count()["customer_id"]

MEMO　class_nameでグループ化して、数を表示する
それだけでは全てのカラムに対する数が出てきてしまうので、代表として["customer_id"]を選択する

In [ ]:
customer_join.groupby("campaign_name").count()["customer_id"]

In [ ]:
customer_join.groupby("gender").count()["customer_id"]

In [ ]:
customer_join.groupby("is_deleted").count()["customer_id"]

In [ ]:
customer_join["start_date"] = pd.to_datetime(customer_join["start_date"])
customer_start = customer_join.loc[customer_join["start_date"]>pd.to_datetime("20180401")]
print(len(customer_start))
customer_start.head()
customer_end_from_20180401 = customer_start.groupby("is_deleted").count()["customer_id"]
customer_end_from_20180401

### ノック24：最新顧客データの基礎集計をしよう

In [ ]:
customer_join["end_date"] = pd.to_datetime(customer_join["end_date"])
customer_newer = customer_join.loc[(customer_join["end_date"]>=pd.to_datetime("20190331"))|(customer_join["end_date"].isna())]
print(len(customer_newer))
customer_newer["end_date"].unique()

MEMO　end_dateが2019年3月31日(退会したユーザ)アンドnullの行を表示

In [ ]:
customer_newer.groupby("class_name").count()["customer_id"]

In [ ]:
customer_newer.groupby("campaign_name").count()["customer_id"]

In [ ]:
customer_newer.groupby("gender").count()["customer_id"]

### ノック25：利用履歴データを集計しよう

In [ ]:
uselog["usedate"] = pd.to_datetime(uselog["usedate"])
uselog["年月"] = uselog["usedate"].dt.strftime("%Y%m")
uselog_months = uselog.groupby(["年月","customer_id"],as_index=False).count()
uselog_months.rename(columns={"log_id":"count"}, inplace=True)
del uselog_months["usedate"]
uselog_months.head()

uselog_months = uselog.groupby(["年月","customer_id"],as_index=False).count()
MEMO groupby を使うと、デフォルトでグループラベルが index になる。index にしたく無い場合は as_index=False を指定する。
年月とcustomer_idでグループ化し、その他のカラムでそれぞれに対する数を表示している

元のオブジェクトを変更: 引数inplace

del = delete

In [ ]:
uselog_customer = uselog_months.groupby("customer_id").agg(["mean", "median", "max", "min"])["count"]
uselog_customer = uselog_customer.reset_index(drop=False)
uselog_customer.head()

GroupBy.mean() のように、グループごとに値を求めて表を作るような操作を Aggregation と呼ぶ。このように GroupBy オブジェクトには Aggregation に使う関数が幾つか定義されているが、これらは agg() を使っても実装出来る。

drop引数

この値は「オリジナルのindexを列として保存しておくならFalse、保存しないならTrue」にします。

### ノック26：利用履歴データから定期利用フラグを作成しよう

In [ ]:
uselog["weekday"] = uselog["usedate"].dt.weekday
uselog_weekday = uselog.groupby(["customer_id","年月","weekday"], as_index=False).count()[["customer_id","年月", "weekday","log_id"]]
uselog_weekday.rename(columns={"log_id":"count"}, inplace=True)
uselog_weekday.head()

In [ ]:
uselog_weekday = uselog_weekday.groupby("customer_id", as_index=False).max()[["customer_id", "count"]]
uselog_weekday["routine_flg"] = 0
uselog_weekday["routine_flg"] = uselog_weekday["routine_flg"].where(uselog_weekday["count"]<4, 1)
uselog_weekday.head()

MEMO　whereは条件に応じて要素の置換を行う

### ノック27：顧客データと利用履歴データを結合しよう

In [ ]:
customer_join = pd.merge(customer_join, uselog_customer, on="customer_id", how="left")
customer_join = pd.merge(customer_join, uselog_weekday[["customer_id", "routine_flg"]], on="customer_id", how="left")
customer_join.head()

In [ ]:
customer_join.isnull().sum()

### ノック28：会員期間を計算しよう

In [ ]:
from dateutil.relativedelta import relativedelta
customer_join["calc_date"] = customer_join["end_date"]
customer_join["calc_date"] = customer_join["calc_date"].fillna(pd.to_datetime("20190430"))
customer_join["membership_period"] = 0
for i in range(len(customer_join)):
    delta = relativedelta(customer_join["calc_date"].iloc[i], customer_join["start_date"].iloc[i])
    customer_join["membership_period"].iloc[i] = delta.years*12 + delta.months
customer_join.head()

MEMO loc=ラベルの取得　iloc=行、列の番号の取得　参考＝http://ailaby.com/lox_iloc_ix/

### ノック29：顧客行動の各種統計量を把握しよう

In [ ]:
customer_join[["mean", "median", "max", "min"]].describe()

In [ ]:
customer_join.groupby("routine_flg").count()["customer_id"]

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.hist(customer_join["membership_period"])

### ノック30：退会ユーザーと継続ユーザーの違いを把握しよう

In [ ]:
customer_end = customer_join.loc[customer_join["is_deleted"]==1]
customer_end.describe()

In [ ]:
customer_stay = customer_join.loc[customer_join["is_deleted"]==0]
customer_stay.describe()

In [ ]:
customer_join.to_csv("customer_join.csv", index=False)